In [1]:
%load_ext autoreload
%autoreload 2
from topgun.optimiser.backtest import BacktestAnalytics
from topgun.reporting import Reporting
import numpy as np
import pandas as pd
import xlwings as xlw

In [2]:
wb = xlw.Book('BACKTEST.xlsm')

# index data from timeseries sheet
Assets = wb.sheets['TIMESERIES'].range('D1').options(pd.DataFrame, expand='table').value.iloc[3:,:].pct_change().dropna()
Assets.index = pd.to_datetime(Assets.index)

In [3]:
# Create a 60/40 portfolio
Assets['60/40']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*((0.6*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global']))

In [4]:
# Create the dataframe that holds the benchmark returns
bmktrtns = Assets[['60/40','STEFI']]
benchmark = '60/40'    # Becnhmark code from benchmarks

In [5]:
# Allocation to Carry Strategy
x1 = 0.02
x2 = 0.04
x3 = 0.06
x4 = 0.08
x5 = 0.10

rtns = pd.DataFrame() # create a empty data frame to store The Returns with allocation to Commodity Strategy

# Calculate the returns : Please note that Carry Strategy is financed from Equity

rtns['E2']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x1)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x1*Assets['Minimum Drawdown Carry']))
rtns['E4']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x2)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x2*Assets['Minimum Drawdown Carry']))
rtns['E6']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x3)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x3*Assets['Minimum Drawdown Carry']))
rtns['E8']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x4)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x4*Assets['Minimum Drawdown Carry']))
rtns['E10']= 0.7*((0.6*Assets['ALSI'])+(0.4*Assets['ALBI']))+0.3*(((0.6-x5)*Assets['MSCI ACWI '])+(0.4*Assets['JP Morgan GBI Global'])+(x5*Assets['Minimum Drawdown Carry']))

In [6]:
bt = BacktestAnalytics(rtns, bmktrtns, bmks_as_rtns=True, benchmark=benchmark, Rf='STEFI')


In [7]:
md=bt.big_bang(title="Backtest 60/40 Portfolio with allocation to Commodity Carry")

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\rolling.py:216: RuntimeWarning:

divide by zero encountered in log

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\rolling.py:216: RuntimeWarning:

divide by zero encountered in log

c:\users\b413863\appdata\local\continuum\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning:

invalid value encountered in double_scalars



In [8]:
Reporting().md2html(md=md, title="Commodity Carry Strategy1")

'REPORT WRITTEN: Commodity Carry Strategy1'